# OWON SPM SCPI Native Serial Communication

This notebook demonstrates how to communicate with an OWON SPM instrument using SCPI commands over `/dev/ttyUSB0` natively (without VISA), using Python's `pyserial` library.

In [ ]:
#!pip install pyserial

import serial

# Configure serial port for OWON SPM (baudrate 115200, adjust timeout as needed)
ser = serial.Serial(
    port='/dev/ttyUSB0',
    baudrate=115200,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=0.5
)

print('Serial port opened:', ser.port)

## 2. Send SCPI Commands to OWON SPM

Write code to send SCPI commands to the OWON SPM device using the serial connection.

In [ ]:
# Send a SCPI command to the OWON SPM (e.g., *IDN?)
def send_scpi_command(ser, command, silent = True):
    # SCPI commands typically end with a newline or carriage return
    ser.write((command + '\n').encode('ascii'))
    if not silent:
        print(f'Sent: {command}')

   
    #response = ser.readline().decode('ascii', errors='replace').strip()
    #print('Received:', response)
# Example: Query instrument identification
send_scpi_command(ser, '*IDN?')

In [ ]:
def read_scpi_response(ser, silent=True):
    # Read until timeout or newline
    response = ser.readline().decode('ascii', errors='replace').strip()
    if not silent:
        print('Received:', response)
    return response


## 4. Display Measurement Data

Display the parsed measurement data in a readable format, such as tables or plots.

In [ ]:
def measure_voltage(ser, command='MEAS:VOLT:DC?'):
    send_scpi_command(ser, command)
    response = read_scpi_response(ser)
    print(f'Voltage Measurement: {response}')
    return response

# Example: Measure DC voltage
voltage = measure_voltage(ser)

In [ ]:

def measure_current(ser, command='MEAS:ALL:INFO?'):
    send_scpi_command(ser, command)
    response = read_scpi_response(ser)
    #print(f'Current Measurement: {response}')
    return response

# Example: Measure DC current
current = measure_current(ser)
print(f'Current Measurement: {current}')

In [ ]:

def extract_current_measurements(text):
    parts = text.split(',')
    floats = [float(x) for x in parts[:3]]
    return floats

# Extract first 3 floats from a comma-separated string
text = '3.214,1.004,3.230,OFF,OFF,OFF,2'
text = measure_current(ser)

In [ ]:
import time
from datetime import datetime
measurements = []
while True:
    current_measurements = extract_current_measurements(measure_current(ser))
    timestamp = datetime.now().isoformat()
    row = [timestamp] + current_measurements
    measurements.append(row)
    #print(row) #formatted output
    print(f'Measurements: {row}')

    time.sleep(1)


In [ ]:
# Set OWON SPM power supply to 3.6V and 1A, then enable output

def set_power_supply(ser, voltage, current):
    # Adjust SCPI commands as needed for your OWON SPM model
    send_scpi_command(ser, f'VOLT {voltage}')
    read_scpi_response(ser)
    send_scpi_command(ser, f'CURR {current}')
    read_scpi_response(ser)
    send_scpi_command(ser, 'OUTP ON')
    read_scpi_response(ser)
    print(f'Set voltage to {voltage} V and current to {current} A, output enabled.')

# Example: Set to 3.6V and 1A, output ON
set_power_supply(ser, 3.6, 3.0)

In [ ]:
send_scpi_command(ser, 'OUTP?')  # Turn off output after use
send_scpi_command(ser, 'CURR:LIM?') 
print(read_scpi_response(ser))
print(read_scpi_response(ser))

In [ ]:
ser.close()  # Close the serial port when don

In [ ]:
import pandas as pd

# Convert measurements to a DataFrame for easier handling
df = pd.DataFrame(measurements, columns=["timestamp"] + [f"measurement_{i}" for i in range(len(measurements[0]) - 1)])
#plot df
import matplotlib.pyplot as plt

df.set_index("timestamp", inplace=True)
df.plot()
plt.show()